In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# Load data

In [2]:
datadir = '/media/gavin/3fa6c02e-278d-413f-ae59-809a756c3966/huawei'
#user_app_usage = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'))

In [4]:
age_train = pd.read_csv(os.path.join(datadir,'age_train.csv'), header=None, names = ['uId','age_group'])

In [5]:
age_train

,uId,age_group
0,1000001,4
1,1000011,3
2,1000015,5
3,1000019,3
4,1000023,2
5,1000025,4
6,1000029,4
7,1000035,2
8,1000037,4
9,1000038,3


In [4]:
age_test = pd.read_csv(os.path.join(datadir,'age_test.csv'), header=None, names = ['uId'])

In [5]:
user_basic_info = pd.read_csv(os.path.join(datadir,'user_basic_info.csv'), header=None, names = ['uId','gender', 'city', 'prodName', 'ramCapacity', 'ramLeftRation', 'romCapacity', 'romLeftRation', 'color', 'fontSize', 'ct', 'carrier', 'os'])

In [6]:
user_basic_info

,uId,gender,city,prodName,ramCapacity,ramLeftRation,romCapacity,romLeftRation,color,fontSize,ct,carrier,os
0,1000001,0,c00145,p00169,3.0,0.43,32.0,0.46,皓月银,1.15000,4g,China_Mobile,8.0
1,1000002,1,c0043,p0022,NaN,0.26,NaN,0.59,渐变黑,1.00000,wifi,China_Mobile,9.0
2,1000004,1,c00281,p0013,3.0,0.36,32.0,0.09,幻海蓝,1.00000,4g#wifi,China_Mobile,8.0
3,1000006,0,c00359,p0045,2.0,0.19,16.0,0.42,灰色,NaN,NaN,China_Mobile,6.0
4,1000007,1,c006,p0097,6.0,0.20,137.0,0.50,宝石蓝,1.00000,4g#wifi,China_Telecom,9.0
5,1000008,1,c0043,p00155,6.0,0.36,128.0,0.47,草木绿,1.15000,4g#wifi,China_Telecom,8.0
6,1000010,0,c0043,p0029,6.0,0.36,64.0,0.01,铂光金,NaN,4g#wifi,China_Mobile,8.0
7,1000011,0,c00126,p0023,NaN,NaN,NaN,NaN,幻夜黑,NaN,4g#wifi,China_Mobile,8.1
8,1000012,0,c00284,p00169,4.0,0.31,64.0,0.39,琥珀金,1.30001,wifi,China_Mobile,8.0
9,1000015,1,c00306,p00169,3.0,0.34,32.0,0.06,皓月银,1.30000,wifi,China_Telecom,8.0


In [7]:
user_behavior_info = pd.read_csv(os.path.join(datadir,'user_behavior_info.csv'), header=None, names = ['uId','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes'])

In [8]:
user_behavior_info

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes
1000001,108,0.00,0.00,1.00,0.07,0.00,0.00,3319
1000002,14,0.00,0.17,4.93,1.23,3.90,1.00,245
1000003,13,1.00,0.00,7.73,3.00,1.70,0.00,5987
1000004,57,0.00,0.03,1.37,0.63,0.00,0.00,7460
1000005,0,0.00,0.00,0.00,0.00,0.00,0.00,0
1000006,33,0.00,0.00,0.00,0.00,0.00,0.00,0
1000007,14,1.50,0.13,2.93,3.13,1.27,0.00,0
1000008,7,0.75,0.07,0.23,3.00,2.43,0.00,29237
1000009,25,7.00,0.07,2.33,2.57,7.63,6.85,29307
1000010,3,0.00,0.00,0.00,0.00,0.00,0.00,2667


In [9]:
user_app_actived = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'), header=None, names = ['uId','appId'])

In [10]:
user_app_actived

,uId,appId
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...
5,1000016,a00102782#a0011894#a002265#a00239261#a00289791...
6,1000021,a00109386#a00145168#a0015359#a0021880#a0023501...
7,1000022,a00109386#a00117437#a00118538#a0013770#a001451...
8,1000036,a00101827#a00101845#a00107264#a00109386#a00116...
9,1000047,a00109386#a00143562#a00200080#a0021880#a002244...


In [11]:
app_info = pd.read_csv(os.path.join(datadir,'app_info.csv'), header=None, names = ['appId','category'])

In [12]:
app_info

,appId,category
0,a006,商务
1,a006,实用工具
2,a0015,影音娱乐
3,a0016,实用工具
4,a0019,动作射击
5,a0024,新闻阅读
6,a0027,教育
7,a0028,教育
8,a0029,便捷生活
9,a0030,便捷生活


In [13]:
#user_app_usage = pd.read_csv(os.path.join(datadir,'user_app_usage.csv'),header=None, names = ['uId','appId','duration','times','use_date'])